<a href="https://colab.research.google.com/github/MinahilSadiq1/Image_Classification_Raw_Pixels_vs._High-Level_Feature_Extractions/blob/main/dogs_vs_cats_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Minahil Sadiq**

**SP20-BCS-023**

**SECTION: B**

# ***ASSIGNMENT 2***

#### the dataset used is taken from kaggle 'dogs vs cats', https://www.kaggle.com/c/dogs-vs-cats

DOWNLOADING AND EXTRACTING DATASET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IMPORTING LIBRARIES

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import zipfile
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage.feature import hog
from scipy.stats import loguniform

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/train.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
len(os.listdir('/tmp/train/'))

25000

1: USING RAW PIXELS AS FEATURES

In [ ]:
# Load the dataset
size = 64
X, y = [], []
for file in tqdm(os.listdir('/tmp/train/')):
    img = cv2.imread(os.path.join('/tmp/train/', file), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue # skip this if it can't be read
    img = cv2.resize(img, (size, size))
    X.append(img.flatten())
    label = 1 if 'dog' in file else 0
    y.append(label)


# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

100%|██████████| 25000/25000 [00:31<00:00, 794.40it/s]


In [ ]:
# Logistic Regression
lr = LogisticRegression(max_iter = 1000, random_state=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)

In [ ]:
# SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(kernel='linear', max_iter=1000, random_state=1000)
svm.fit(X_train_scaled, y_train)
svm_pred = svm.predict(X_test_scaled)
svm_acc = accuracy_score(y_test, svm_pred)

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=1000)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)

In [ ]:
print("Logistic Regression Accuracy: {:.2f}%".format(lr_acc * 100))
print("SVM Accuracy: {:.2f}%".format(svm_acc * 100))
print("Random Forest Accuracy: {:.2f}%".format(rf_acc * 100))

Logistic Regression Accuracy: 52.54%
SVM Accuracy: 51.42%
Random Forest Accuracy: 63.54%


2: EXTRACT HIGH-LEVEL FEATURES AT LEAST 2 HIGH-LEVEL FEATURE TECHNIQUES, eg hog, orb etc

In [ ]:
# HOG ----------------------------
size = 64
def extract_hog_features(image):
    features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), transform_sqrt=True, feature_vector=True)
    return features

X_hog, y_hog = [], []
for file in tqdm(os.listdir('/tmp/train/')):
    img = cv2.imread(os.path.join('/tmp/train/', file), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue # skip this file if it can't be read
    img = cv2.resize(img, (size, size))
    features = extract_hog_features(img)
    X_hog.append(features)
    label = 1 if 'dog' in file else 0
    y_hog.append(label)

X_hog = np.array(X_hog)
y_hog = np.array(y_hog)

#SPLITTING
X_hog_train, X_hog_test, y_hog_train, y_hog_test = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)


100%|██████████| 25000/25000 [01:41<00:00, 246.52it/s]


have done the below code block to solve issue of 'SURF' but it did not get resolved.

In [ ]:
# ORB --------------------------
def extract_orb_features(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        descriptors = np.zeros((1, 32))
    return descriptors.flatten()


# Histogram of ORB features
def orb_histogram(image):
    features = extract_orb_features(image)
    hist, _ = np.histogram(features, bins=16, range=(0, 255))
    return hist

X_orb, y_orb = [], []
for file in tqdm(os.listdir('/tmp/train/')):
    img = cv2.imread(os.path.join('/tmp/train/', file), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    hist = orb_histogram(img)
    X_orb.append(hist)
    label = 1 if 'dog' in file else 0
    y_orb.append(label)

X_orb = np.array(X_orb)
y_orb = np.array(y_orb)

# Train-test split
X_orb_train, X_orb_test, y_orb_train, y_orb_test = train_test_split(X_orb, y_orb, test_size=0.2, random_state=42)


100%|██████████| 25000/25000 [02:39<00:00, 156.97it/s]


In [ ]:
# after hog, orb .. now training three models SVM, LOGISTIC REGRESSION, RANDOM FOREST

# Logistic Regression
#HOG
lr_hog = LogisticRegression(max_iter=1000, random_state=42)
lr_hog.fit(X_hog_train, y_hog_train)
lr_hog_pred = lr_hog.predict(X_hog_test)
lr_hog_acc = accuracy_score(y_hog_test, lr_hog_pred)

#ORB
lr_orb = LogisticRegression(max_iter=1000, random_state=42)
lr_orb.fit(X_orb_train, y_orb_train)
lr_orb_pred = lr_orb.predict(X_orb_test)
lr_orb_acc = accuracy_score(y_orb_test, lr_orb_pred)

In [ ]:
# SVM
#HOG
scaler = StandardScaler()
X_hog_train_scaled = scaler.fit_transform(X_hog_train)
X_hog_test_scaled = scaler.transform(X_hog_test)

svm_hog = SVC(kernel='linear', max_iter=1000, random_state=42)
svm_hog.fit(X_hog_train_scaled, y_hog_train)
svm_hog_pred = svm_hog.predict(X_hog_test_scaled)
svm_hog_acc = accuracy_score(y_hog_test, svm_hog_pred)

#ORB
scaler = StandardScaler()
X_orb_train_scaled = scaler.fit_transform(X_orb_train)
X_orb_test_scaled = scaler.transform(X_orb_test)

svm_orb = SVC(kernel='linear',max_iter=1000, random_state=42)
svm_orb.fit(X_orb_train_scaled, y_orb_train)
svm_orb_pred = svm_orb.predict(X_orb_test_scaled)
svm_orb_acc = accuracy_score(y_orb_test, svm_orb_pred)

In [ ]:
# Random Forest
#HOG
rf_hog = RandomForestClassifier(n_estimators=100, random_state=42)
rf_hog.fit(X_hog_train, y_hog_train)
rf_hog_pred = rf_hog.predict(X_hog_test)
rf_hog_acc = accuracy_score(y_hog_test, rf_hog_pred)

#ORB
rf_orb = RandomForestClassifier(n_estimators=100, random_state=42)
rf_orb.fit(X_orb_train, y_orb_train)
rf_orb_pred = rf_orb.predict(X_orb_test)
rf_orb_acc = accuracy_score(y_orb_test, rf_orb_pred)

In [ ]:
#accuracy score of logistic regression
print("Logistic Regression (HOG) Accuracy: {:.2f}%".format(lr_hog_acc * 100))
print("Logistic Regression (ORB) Accuracy: {:.2f}%".format(lr_orb_acc * 100))
print()
#accuracy score of svm
print("SVM (HOG) Accuracy: {:.2f}%".format(svm_hog_acc * 100))
print("SVM (ORB) Accuracy: {:.2f}%".format(svm_orb_acc * 100))
print()
#accuracy score of randomforest
print("RANDOM FOREST (HOG) Accuracy: {:.2f}%".format(rf_hog_acc * 100))
print("RANDOM FOREST (ORB) Accuracy: {:.2f}%".format(rf_orb_acc * 100))
print()

Logistic Regression (HOG) Accuracy: 72.30%
Logistic Regression (ORB) Accuracy: 63.62%

SVM (HOG) Accuracy: 55.26%
SVM (ORB) Accuracy: 44.84%

RANDOM FOREST (HOG) Accuracy: 69.78%
RANDOM FOREST (ORB) Accuracy: 63.74%



3: OPTIMIZE HYPERPARAMETERS

In [ ]:
# Logistic Regression Hyperparameter Tuning
#HOG
lr_params = {'C': [0.01, 0.1, 1, 10]}
lr_hog_grid = GridSearchCV(lr_hog, lr_params, cv=5)
lr_hog_grid.fit(X_hog_train, y_hog_train)
lr_hog_best = lr_hog_grid.best_estimator_
lr_hog_best_pred = lr_hog_best.predict(X_hog_test)
lr_hog_best_acc = accuracy_score(y_hog_test, lr_hog_best_pred)

#ORB
lr_orb_grid = GridSearchCV(lr_orb, lr_params, cv=5)
lr_orb_grid.fit(X_orb_train, y_orb_train)
lr_orb_best = lr_orb_grid.best_estimator_
lr_orb_best_pred = lr_orb_best.predict(X_orb_test)
lr_orb_best_acc = accuracy_score(y_orb_test, lr_orb_best_pred)


In [ ]:
# SVM Hyperparameter Tuning
# define the hyperparameter search space
svm_params = {'C': [0.01, 0.1, 1, 10], 'kernel': ['rbf', 'linear',]}
#HOG
svm_hog_grid = RandomizedSearchCV(
    svm_hog,
    param_distributions=svm_params,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    random_state=42
)
svm_hog_grid.fit(X_hog_train_scaled, y_hog_train)
svm_hog_best = svm_hog_grid.best_estimator_
svm_hog_best_pred = svm_hog_best.predict(X_hog_test_scaled)
svm_hog_best_acc = accuracy_score(y_hog_test, svm_hog_best_pred)

#ORB
svm_orb_grid = RandomizedSearchCV(svm_orb, param_distributions=svm_params,
    n_iter=1000,
    cv=5,
    n_jobs=-1,
    random_state=42)
svm_orb_grid.fit(X_orb_train_scaled, y_orb_train)
svm_orb_best = svm_orb_grid.best_estimator_
svm_orb_best_pred = svm_orb_best.predict(X_orb_test_scaled)
svm_orb_best_acc = accuracy_score(y_orb_test, svm_orb_best_pred)

In [ ]:
# Random Forest Hyperparameter Tuning
# Define the hyperparameter search space
rf_params = {'n_estimators': [10, 50], 'max_depth': [None, 100, 200, 300], 'min_samples_split': [300, 600, 1000]}
# Create the random forest classifier
rf_hogg = RandomForestClassifier(random_state=42)
# Create the randomized search object
rf_hog_random = RandomizedSearchCV(
    estimator=rf_hogg,
    param_distributions=rf_params,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    random_state=42
)
# Fit the randomized search object to the training data
rf_hog_random.fit(X_hog_train, y_hog_train)

rf_hog_best = rf_hog_random.best_estimator_
rf_hog_best_pred = rf_hog_best.predict(X_hog_test)
rf_hog_best_acc = accuracy_score(y_hog_test, rf_hog_best_pred)


# Create the random forest classifier
rf_orbb = RandomForestClassifier(random_state=42)
# Create the randomized search object
rf_orb_random = RandomizedSearchCV(
    estimator=rf_orbb,
    param_distributions=rf_params,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    random_state=42
)

rf_orb_random.fit(X_orb_train, y_orb_train)
rf_orb_best = rf_orb_random.best_estimator_
rf_orb_best_pred = rf_orb_best.predict(X_orb_test)
rf_orb_best_acc = accuracy_score(y_orb_test, rf_orb_best_pred)


In [ ]:
# Print results
print("Logistic Regression Results:")
print(f"HOG Features: {lr_hog_best_acc * 100} with {lr_hog_best.get_params()}")
print(f"ORB Features: {lr_orb_best_acc * 100} with {lr_orb_best.get_params()}\n")

print("SVM Results:")
print(f"HOG Features: {svm_hog_best_acc * 100} with {svm_hog_best.get_params()}")
print(f"ORB Features: {svm_orb_best_acc * 100} with {svm_orb_best.get_params()}\n")

print("Random Forest Results:")
print(f"HOG Features: {rf_hog_best_acc * 100} with {rf_hog_best.get_params()}")
print(f"ORB Features: {rf_orb_best_acc * 100} with {rf_orb_best.get_params()}")

Logistic Regression Results:
HOG Features: 72.42 with {'C': 0.1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
ORB Features: 62.38 with {'C': 0.01, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

SVM Results:
HOG Features: 63.46000000000001 with {'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 1000, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}
ORB Features: 56.279999999999994 with {

# **Methods and Results**
First import all the required Libraries, then load data from the drive.

**Part 1:** raw pixels used as features, we chose 3 classifiers *Logistic Regression*, *SVM*, *RandomForest*.after that applied a *for loop* in which we covert the labels of images into the binary form(0,1). Dog images labeled as 1 and cat images labeled as 0. we divide the data into training and testing with the ratio of 80:20.

For first model *Logistic Regression*, we set maximum iterations to 1000 as our data is of 25000 images. we trained the model using .fit method. and then we test it on unseen data. it provides us the accuracy of **52.54%**.

For *SVM*, we set max_iter to 1000, and used *linear kernel* trained and test the model and it gives the accuracy of **51.42%**.

For *RandomForest* we set n_estimators to 100 as decreasing no. of it will make the training faster. after training the model, we test it and it gives the accuracy of **63.54%**.

**Part 2:** extracting high level features using techniques such as hog, orb.

*Hog*: function 'extract_hog_features'takes input image and create Histogram of Oriented Gradients (HOG) features using 'hog' function. the parameters used in this are orientations, pixels_per_cell etc.. then we make two arrays x_hog and y_hog which store the hog features of image and the resultanat labels of it respectively. resizing image and then function created above will be used on it. then it will label the data in binary form which will make it able to work for classifiers.

*Orb*: just like Hog but it will definitely use its own properties and parameters but working is same like Hog except where we use the hog now will use Orb.

For *Logistic Regression* it gives us the accuracy of **72.30%** with **HOG** feature technique.

For *Logistic Regression* it give us accuracy of **63.62%** with **ORB** feature technique.

For *SVM* it give accuracy **55.26%** using **HOG** fetaure tecnique.

For *SVM* it give accuracy **44.84%** using **ORB**.

For *RandomForest* accuracy= **69.78%** using **HOB**.

For *RadomForest* accuracy= **63.74%** using **ORB**.

**Part 3:** Optimizing hyperparameters

*logistic regression model* to classify images of dogs and cats. It is trying different values of a hyperparameter called 'C' to find the best value for the model. The 'GridSearchCV' function is used to search through a grid of possible 'C' values and find the one that gives the highest accuracy. The best model is then used to predict the labels of a test set of images. It gives **72.42%** accuracy with the best parameter among all **c=0.1** using "HOG".

*logistic regression model* using "ORB" provide accuracy **62.38%** with **c=0.01**.

For *SVM* two kernels used first linear nad poly in which it returns linear for both HOG and orb.

*SVM* using "HOG" accuracy=**59.0%** with **c=0.01, kernel= linear**.

*SVM* using "orb" accuracy=**56.26%** with **c=10, kernel= linear**.

Then we use other two kernels linear and rbf then it returns rbf as best for both.

*SVM* using "HOG" accuracy=**63.4%** with **c=1, kernel= rbf**.

*SVM* using "orb" accuracy=**56.26%** with **c=0.1, kernel= rbf**.

*RandomForest* "HOG" accuracy=**69.5%** with **max depth = 200, max feature= sqrt, min samples split= 300**.

*RandomForest* "ORB" accuracy=**63.68%** with **max depth = 200, max feature= sqrt, min samples split= 300**.
